理解随机过程的含义

赌博中的随机过程

赌博的例子非常普遍，这里我们举一个大家非常熟悉的例子，我们尽量挖掘的更深入一些。

赌徒和庄家对赌抛硬币，如果为正面，本轮赌徒赢，庄家付给赌徒1元，结果为反面，本轮赌徒输，赌徒付给庄家1元。赌徒有初始赌本10元，手上的钱一旦输光则退出赌局，如何来模拟这个赌博过程？

我们首先来分析一下这个过程，赌徒的赌博结果本质上依托于每次抛掷硬币的结果，每一轮赌博就是一个伯努利试验，赢的概率是p=0.5，赌博的过程就是由这一串伯努利试验构成的伯努利随机过程，每轮赌局中，如果赢则赌本增加1元，输则赌本减少1元。

当然了，如果对某一个特定的赌徒，一旦开始进入赌局，则最终由每轮赌局结果构成的序列就是唯一的。那么如果我们想观察整个赌博过程的整体特征，我们该怎么办？好办，还是使用之前讲过的蒙特卡罗方法，采用大量的样本，最终观察样本结果的整体特征。

这里，我们为了说明问题，先采用的样本数也就是赌徒数为10个，轮数为100轮，也就是每个赌徒最多和庄家对赌100轮，如果在这个过程中输光了赌本，则提前退出，如果到100轮还有赌本，赌局也停止。

我们来看一下模拟的代码：

In [5]:
import pandas as pd
import random

sample_list = []
round_num =100
person_num = 10
for person in range (1,person_num + 1):
    money = 10 #本金十块
    for round in range(1,round_num + 1):
        result = random.randint(0, 1) #随机输赢
        if result == 1:
            money = money + 1
        elif result == 0:
            money = money - 1
        if money == 0:
            break
    sample_list.append([person,round,money])


sample_df = pd.DataFrame(sample_list,columns = ['person','round','money'])
sample_df.set_index('person',inplace=True)


# print(sample_df)
pd.DataFrame(sample_df)

,round,money
person,,
1,100,20
2,22,0
3,100,14
4,100,12
5,100,14
6,42,0
7,32,0
8,100,8
9,100,8


我们简单的分析一些代码和运行结果，其实这段代码中最核心的部分就是：

result = random.randint(0, 1)

在每轮赌局中，我们首先生成了一个随机变量，他在0和1当中等概论选取，模拟的就是抛掷硬币的过程，当结果为1时表示硬币为正面，本轮赢庄家1元，结果为0时表示硬币为反面，本轮输庄家1元。这里设定的是每个赌徒最多赌100轮，如果不到100轮就输光了，则退场，这样就模拟了整个赌博的随机过程。

从运行结果来看，10个赌徒中有  个提前输光退场，剩下的 个打满全场的人中，有 个是挣钱的，有 个是亏钱的。

当然，我们这里是为了打印出所有的结果，所以样本数选择的比较少。蒙特卡洛方法讲求大的样本量，我们把样本数和轮数都修改一下，并且统计一些指标。我们把赌徒的总人数设置为1000000人，轮数设置为100（我们依次修改了轮数为100,1000,10000），来观察样本总体的表现：

代码片段：

In [8]:
import pandas as pd
import random

sample_list = []
person_num = 100000
round_num = 10000
for person in range(1, person_num + 1):
    money = 10
    for round in range(1, round_num + 1):
        result = random.randint(0, 1)
        if result == 1:
            money = money + 1
        elif result == 0:
            money = money - 1
        if money == 0:
            break
    sample_list.append([person, round, money])
sample_df = pd.DataFrame(sample_list, columns=['person', 'round', 'money'])
sample_df.set_index('person',inplace=True)

print("总轮数：{}，总人数{}".format(round_num,person_num))
print('输光读本提前出局的人数：{}'.format(person_num-len(sample_df[sample_df['round']==round_num])))
print("赌满全场且盈利的人数:{}".format(len(sample_df[sample_df['money']>10])))
print("赌满全场且亏损的人数:{}".format(len(sample_df[sample_df['money']<=10][sample_df['money']>0])))

hon-input-8-72d6bc22725d>:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print("赌满全场且亏损的人数:{}".format(len(sample_df[sample_df['money']<=10][sample_df['money']>0])))


从结果中不难发现，这种和庄家1:1的对赌，随着轮数的增加，基本上都破产被收割了。换句话说，哪怕庄家不出千，输赢概率各半，赌的越久，基本上都是输光破产走人，原因是什么？原因是庄家的资金量是无穷的。

2.2.模拟股价的变化

赌博的例子结束了，我们再举一个股票的例子，这个让人感觉都很心跳啊，在金融工程中，有下面 这样一个公式，他利用目前的股价 $S_{t}$ 去预测 $\Delta t$ 时间之后的股价 $S_{t+1}$ :
$$
S_{t+1}=S_{t}+\hat{\mu} S_{t} \Delta t+\sigma S_{t} \epsilon \sqrt{\Delta t}
$$
这其中的参数我来解释一下：
$\hat{\mu}$ 表示股票收益率的期望值，这里我们设定为 $15 \%,$ 即 $\hat{\mu}=0.15$
$\sigma$ 表示股票的波动率，这里设定为 $\sigma=0.2$
$\Delta t=\frac{T}{n},$ 其中 $T$ 表示整数年份, $n$ 表示在整个估算周期内，取的具体步数，就好比说 $T$ 为 一年, $n$ 如果取244，那么 $\Delta t$ 的粒度就是每个交易日了（一年有244个交易日）。
这里面似乎所有的参数都是确定的，唯独除了 $\epsilon$ 之外, $\epsilon$ 是一个服从标准正态分布的随机变量, 这是这个 $\epsilon,$ 决定了每日的股价 $S_{i}$ 是一个随机变量，而由股价构成的序列是一个随机过程。
我们同样的用蒙特卡罗方法，利用大样本来估计一下在目前股价为 $S_{0}=10$ 的情况下，1年之后股 价的概率分布情况。